In [ ]:
import cv2
import numpy as np
import os

class Cartoonizer:
    def __init__(self, resize_dim: tuple = (800, 800), blur_kernel: int = 5, 
                 edge_preset: str = 'medium', color_preset: str = 'medium'):
        """
        Initialize Cartoonizer with configurable parameters.
        """
        self.resize_dim = resize_dim
        self.blur_kernel = blur_kernel
        self.edge_presets = {
            'soft': {'block_size': 7, 'c': 7, 'canny_thresh': (50, 150)},
            'medium': {'block_size': 9, 'c': 9, 'canny_thresh': (100, 200)},
            'strong': {'block_size': 11, 'c': 11, 'canny_thresh': (150, 250)}
        }
        self.color_presets = {
            'soft': {'d': 5, 'sigma_color': 50, 'sigma_space': 50},
            'medium': {'d': 7, 'sigma_color': 100, 'sigma_space': 100},
            'strong': {'d': 9, 'sigma_color': 150, 'sigma_space': 150}
        }
        self.edge_params = self.edge_presets.get(edge_preset, self.edge_presets['medium'])
        self.color_params = self.color_presets.get(color_preset, self.color_presets['medium'])

    def _adaptive_resize(self, image: np.ndarray) -> np.ndarray:
        """Maintain aspect ratio while resizing"""
        if image is None:
            raise ValueError("Cannot resize a None image.")
        
        h, w = image.shape[:2]
        scale = min(self.resize_dim[0]/w, self.resize_dim[1]/h)
        return cv2.resize(image, (int(w*scale), int(h*scale)), interpolation=cv2.INTER_AREA)

    def _enhance_edges(self, gray: np.ndarray) -> np.ndarray:
        """Advanced edge detection"""
        blurred = cv2.medianBlur(gray, self.blur_kernel)
        edges = cv2.adaptiveThreshold(
            blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY_INV, self.edge_params['block_size'], 
            self.edge_params['c']
        )
        canny_edges = cv2.Canny(blurred, *self.edge_params['canny_thresh'])
        combined_edges = cv2.bitwise_or(edges, canny_edges)
        
        # Morphological closing to enhance edge continuity
        kernel = np.ones((3,3), np.uint8)
        return cv2.morphologyEx(combined_edges, cv2.MORPH_CLOSE, kernel)

    def _smooth_colors(self, image: np.ndarray) -> np.ndarray:
        """Advanced color smoothing"""
        smoothed = cv2.bilateralFilter(
            image, d=self.color_params['d'],
            sigmaColor=self.color_params['sigma_color'],
            sigmaSpace=self.color_params['sigma_space']
        )
        Z = smoothed.reshape((-1, 3))
        Z = np.float32(Z)
        
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 1.0)
        _, label, center = cv2.kmeans(
            Z, K=8, bestLabels=None,
            criteria=criteria, attempts=10,
            flags=cv2.KMEANS_RANDOM_CENTERS
        )
        
        center = np.uint8(center)
        quantized = center[label.flatten()]
        return quantized.reshape(smoothed.shape)

    def _combine_layers(self, edges: np.ndarray, colors: np.ndarray) -> np.ndarray:
        """Combine edge and color layers"""
        edges_inv = cv2.bitwise_not(edges)
        edges_col = cv2.cvtColor(edges_inv, cv2.COLOR_GRAY2BGR)
        cartoon = cv2.bitwise_and(colors, edges_col)
        
        return cartoon

    def process_image(self, image_path: str) -> np.ndarray:
        """Full processing pipeline"""
        try:
            # Validate image path
            if not os.path.exists(image_path):
                raise FileNotFoundError(f"Error: Image not found at {image_path}")
            
            image = cv2.imread(image_path)
            if image is None:
                raise ValueError(f"Error: Failed to load image from {image_path}")
            
            image = self._adaptive_resize(image)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            edges = self._enhance_edges(gray)
            colors = self._smooth_colors(image)
            
            cartoon = self._combine_layers(edges, colors)
            
            # Post-processing filters
            cartoon = cv2.detailEnhance(cartoon, sigma_s=10, sigma_r=0.15)
            cartoon = cv2.edgePreservingFilter(cartoon, flags=1, sigma_s=64, sigma_r=0.2)
            
            return cartoon
        
        except Exception as e:
            print(f"Error processing image: {str(e)}")
            return None

    def show_comparison(self, original: np.ndarray, processed: np.ndarray):
        """Display before/after comparison"""
        if original is None or processed is None:
            print("Error: One of the images is None, cannot display comparison.")
            return
        
        if original.shape != processed.shape:
            processed = cv2.resize(processed, (original.shape[1], original.shape[0]))
            
        comparison = np.hstack((original, processed))
        cv2.imshow('Before / After', comparison)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

if __name__ == "__main__":
    # Example usage
    input_path = "img/h2.jpeg"  # Change this to an actual image path
    
    cartoonizer = Cartoonizer(resize_dim=(1200, 1200), edge_preset='strong', color_preset='medium')
    
    result = cartoonizer.process_image(input_path)
    
    if result is not None:
        original = cv2.imread(input_path)
        original = cartoonizer._adaptive_resize(original)
        cartoonizer.show_comparison(original, result)
        cv2.imwrite("cartoon_output2.jpg", result)
    else:
        print("Failed to generate cartoon effect.")
